In [1]:
#
# LSTM모델을 이용해서 주가예측
#
import os
import settings
import pandas as pd        # 라이브러리
import numpy as np         # 라이브러리
from keras.models import Sequential      # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import Dense           # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import LSTM            # 딥러닝을 구동하는 데 필요한 케라스 함수
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from sklearn.metrics import mean_squared_error
import pymysql         # 파이썬에서 mysql연동시켜주는 라이브러리

#
# DB테이블 값 조회 (SELECT)
#
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')

# 오차율 가져오기
try:
    with connection.cursor() as cursor:
        sql = "select accuracy from stock_hye WHERE company_name='LGD'"
        cursor.execute("set names utf8")
        cursor.execute(sql)
        result_accuracy = cursor.fetchone()
        
        for i in result_accuracy:
            accuracy = i
            
finally:
    connection.close()


# 데이터셋 생성 함수
look_back = 1
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
 
#
# 저장되어있는 주식데이터 불러오기
#
sydtpath = os.path.join(settings.BASE_DIR, 'chart_data/%s' % (settings.get_today_str()))
stock_code = "lgdisplay"
fullpath = sydtpath + os.path.sep + stock_code + '.csv'
pandf = pd.read_csv(fullpath, index_col="Date")

# 데이터 전처리
nparr = pandf['Close'].values[1:]     # 맨처음 'Close'데이터부터 차례대로 nparr에 저장
print(nparr)
nparr.astype('float32')    # float형으로 변환
print(nparr)
nparr = nparr.reshape(-1,1)
print(nparr)
 
# 정규화 (0~1사이의 값으로 바꿔준다)
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
 
# 학습용, 테스트용 데이터로 나누기 (90%를 학습용 데이터, 10%를 테스트용 데이터)
train_size = int(len(nptf) * 0.9)
test_size = len(nptf) - train_size
train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
print(len(train), len(test))
 
# 학습을 위한 데이터셋 생성하기 (학습용, 테스트용으로 구분)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# RNN모델은 3차원 데이터
# trainX, testX값을 [samples, time steps, features] 형태로 reshape
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
 
# LSTM모델
model = Sequential()                                                # 딥러닝 구조, 층을 설정
model.add(LSTM(20, input_shape=(1, 1)))     # (timestep, feature)   # 층이 추가됨 (add)   # 입력층, 첫번째 은닉층                   
model.add(Dense(1))                                                 # 출력층 (하나)
# 모델을 컴파일 (컴퓨터가 알아들을 수 있도록)   # 오차함수, 최적화 방법
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])   # metrics : 모델 수행 결과를 나타내게 설정 (과적합 문제 방지)
model.fit(trainX, trainY, epochs=1000, batch_size=50, verbose=2)      # 모델을 실제로 수행     # batch_size : 전체 데이터를 10개씩 사용
                                                                                               # verbose(로깅)  2 : epoch당 나오게

# 예측값 평가하기 (얼마나 정확한지)
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)                 # testPredict : 예측 값
testY = scaler.inverse_transform(testY)                             # testY : 실제 값
testScore = math.sqrt(mean_squared_error(testY, testPredict))       # mean_squared_error : 평균 제곱근 오차
print('Train Score: %.2f RMSE' % testScore)                         # 예측 값과 실제 값 차이 출력
 
# 예측 데이터 출력
lastX = nptf[-1]
lastX = np.reshape(lastX, (1, 1, 1))
lastY = model.predict(lastX)
predict = scaler.inverse_transform(lastY)                    # 정규화 시킨 값을 역변환
print('Predict the Close value of final day: %d' % predict)  # 데이터 입력 마지막 다음날 종가 예측
    
# 차트출력, 저장
font_path = "C:/Windows/Fonts/a옛날목욕탕L.ttf"
fontprop = fm.FontProperties(fname=font_path, size=15)

plt.plot(testPredict)
plt.plot(testY)

plt.title('[LG 디스플레이] 예측 그래프\n예측값 : %d \t\t\t\t\t\t\t\t오차율 : %s' % (predict, accuracy), fontproperties=fontprop)

#plt.savefig("./chart_picture/lgdisplay.png",dpi=300)
plt.savefig("C:\source\SPF\chart_picture\lgdisplay.png", dpi=300)
#plt.show()

# 사진 데이터 binary형식으로 바꿔주는 함수
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


# 
# DB테이블 값 삽입 (INSERT)
#
# MySQL Connection 연결
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')
try:
    with connection.cursor() as cursor:
        sql = 'INSERT INTO stock_hye (company_name, stock_price, image) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE stock_price = VALUES(stock_price), image = VALUES(image)'
        image = convertToBinaryData("C:\source\SPF\chart_picture\lgdisplay.png")
        cursor.execute(sql, ('LGD', int(predict), image))          # 넣으려는 값
    connection.commit()
    
finally:
    connection.close()

Using TensorFlow backend.


[24400 25250 25600 ... 17200 17200 16750]
[24400 25250 25600 ... 17200 17200 16750]
[[24400]
 [25250]
 [25600]
 ...
 [17200]
 [17200]
 [16750]]
2308 257


C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/1000
 - 1s - loss: 0.1343 - acc: 0.0000e+00
Epoch 2/1000
 - 0s - loss: 0.0445 - acc: 0.0000e+00
Epoch 3/1000
 - 0s - loss: 0.0139 - acc: 4.3365e-04
Epoch 4/1000
 - 0s - loss: 0.0100 - acc: 4.3365e-04
Epoch 5/1000
 - 0s - loss: 0.0091 - acc: 4.3365e-04
Epoch 6/1000
 - 0s - loss: 0.0081 - acc: 4.3365e-04
Epoch 7/1000
 - 0s - loss: 0.0072 - acc: 4.3365e-04
Epoch 8/1000
 - 0s - loss: 0.0063 - acc: 4.3365e-04
Epoch 9/1000
 - 0s - loss: 0.0054 - acc: 4.3365e-04
Epoch 10/1000
 - 0s - loss: 0.0045 - acc: 4.3365e-04
Epoch 11/1000
 - 0s - loss: 0.0037 - acc: 4.3365e-04
Epoch 12/1000
 - 0s - loss: 0.0030 - acc: 4.3365e-04
Epoch 13/1000
 - 0s - loss: 0.0024 - acc: 4.3365e-04
Epoch 14/1000
 - 0s - loss: 0.0018 - acc: 4.3365e-04
Epoch 15/1000
 - 0s - loss: 0.0014 - acc: 4.3365e-04
Epoch 16/1000
 - 0s - loss: 0.0011 - acc: 4.3365e-04
Epoch 17/1000
 - 0s - loss: 8.7441e-04 - acc: 4.3365e-04
Epoch 18/1000
 - 0s - loss: 7.1682e-04 - acc: 4.3365e-04
Epoch 19/1000
 - 0s - loss: 6.1516e-04 - acc: 4

 - 0s - loss: 4.5287e-04 - acc: 4.3365e-04
Epoch 146/1000
 - 0s - loss: 4.3273e-04 - acc: 4.3365e-04
Epoch 147/1000
 - 0s - loss: 4.3610e-04 - acc: 4.3365e-04
Epoch 148/1000
 - 0s - loss: 4.3283e-04 - acc: 4.3365e-04
Epoch 149/1000
 - 0s - loss: 4.3182e-04 - acc: 4.3365e-04
Epoch 150/1000
 - 0s - loss: 4.4143e-04 - acc: 4.3365e-04
Epoch 151/1000
 - 0s - loss: 4.3354e-04 - acc: 4.3365e-04
Epoch 152/1000
 - 0s - loss: 4.4701e-04 - acc: 4.3365e-04
Epoch 153/1000
 - 0s - loss: 4.3573e-04 - acc: 4.3365e-04
Epoch 154/1000
 - 0s - loss: 4.3383e-04 - acc: 4.3365e-04
Epoch 155/1000
 - 0s - loss: 4.3020e-04 - acc: 4.3365e-04
Epoch 156/1000
 - 0s - loss: 4.3011e-04 - acc: 4.3365e-04
Epoch 157/1000
 - 0s - loss: 4.3262e-04 - acc: 4.3365e-04
Epoch 158/1000
 - 0s - loss: 4.3284e-04 - acc: 4.3365e-04
Epoch 159/1000
 - 0s - loss: 4.3597e-04 - acc: 4.3365e-04
Epoch 160/1000
 - 0s - loss: 4.3528e-04 - acc: 4.3365e-04
Epoch 161/1000
 - 0s - loss: 4.2933e-04 - acc: 4.3365e-04
Epoch 162/1000
 - 0s - loss: 

...

 - 0s - loss: 4.4620e-04 - acc: 4.3365e-04
Epoch 429/1000
 - 0s - loss: 4.3460e-04 - acc: 4.3365e-04
Epoch 430/1000
 - 0s - loss: 4.3159e-04 - acc: 4.3365e-04
Epoch 431/1000
 - 0s - loss: 4.3351e-04 - acc: 4.3365e-04
Epoch 432/1000
 - 0s - loss: 4.3687e-04 - acc: 4.3365e-04
Epoch 433/1000
 - 0s - loss: 4.3544e-04 - acc: 4.3365e-04
Epoch 434/1000
 - 0s - loss: 4.3107e-04 - acc: 4.3365e-04
Epoch 435/1000
 - 0s - loss: 4.4544e-04 - acc: 4.3365e-04
Epoch 436/1000
 - 0s - loss: 4.3496e-04 - acc: 4.3365e-04
Epoch 437/1000
 - 0s - loss: 4.3537e-04 - acc: 4.3365e-04
Epoch 438/1000
 - 0s - loss: 4.3366e-04 - acc: 4.3365e-04
Epoch 439/1000
 - 0s - loss: 4.3280e-04 - acc: 4.3365e-04
Epoch 440/1000
 - 0s - loss: 4.3818e-04 - acc: 4.3365e-04
Epoch 441/1000
 - 0s - loss: 4.3023e-04 - acc: 4.3365e-04
Epoch 442/1000
 - 0s - loss: 4.3397e-04 - acc: 4.3365e-04
Epoch 443/1000
 - 0s - loss: 4.3071e-04 - acc: 4.3365e-04
Epoch 444/1000
 - 0s - loss: 4.4419e-04 - acc: 4.3365e-04
Epoch 445/1000
 - 0s - loss: 

Epoch 570/1000
 - 0s - loss: 4.3457e-04 - acc: 4.3365e-04
Epoch 571/1000
 - 0s - loss: 4.3444e-04 - acc: 4.3365e-04
Epoch 572/1000
 - 0s - loss: 4.3418e-04 - acc: 4.3365e-04
Epoch 573/1000
 - 0s - loss: 4.3370e-04 - acc: 4.3365e-04
Epoch 574/1000
 - 0s - loss: 4.3199e-04 - acc: 4.3365e-04
Epoch 575/1000
 - 0s - loss: 4.2848e-04 - acc: 4.3365e-04
Epoch 576/1000
 - 0s - loss: 4.3173e-04 - acc: 4.3365e-04
Epoch 577/1000
 - 0s - loss: 4.3451e-04 - acc: 4.3365e-04
Epoch 578/1000
 - 0s - loss: 4.2980e-04 - acc: 4.3365e-04
Epoch 579/1000
 - 0s - loss: 4.3705e-04 - acc: 4.3365e-04
Epoch 580/1000
 - 0s - loss: 4.3074e-04 - acc: 4.3365e-04
Epoch 581/1000
 - 0s - loss: 4.3286e-04 - acc: 4.3365e-04
Epoch 582/1000
 - 0s - loss: 4.3262e-04 - acc: 4.3365e-04
Epoch 583/1000
 - 0s - loss: 4.3503e-04 - acc: 4.3365e-04
Epoch 584/1000
 - 0s - loss: 4.4007e-04 - acc: 4.3365e-04
Epoch 585/1000
 - 0s - loss: 4.2937e-04 - acc: 4.3365e-04
Epoch 586/1000
 - 0s - loss: 4.3680e-04 - acc: 4.3365e-04
Epoch 587/1000

 - 0s - loss: 4.3111e-04 - acc: 4.3365e-04
Epoch 712/1000
 - 0s - loss: 4.3959e-04 - acc: 4.3365e-04
Epoch 713/1000
 - 0s - loss: 4.3183e-04 - acc: 4.3365e-04
Epoch 714/1000
 - 0s - loss: 4.4334e-04 - acc: 4.3365e-04
Epoch 715/1000
 - 0s - loss: 4.3011e-04 - acc: 4.3365e-04
Epoch 716/1000
 - 0s - loss: 4.3327e-04 - acc: 4.3365e-04
Epoch 717/1000
 - 0s - loss: 4.4189e-04 - acc: 4.3365e-04
Epoch 718/1000
 - 0s - loss: 4.4333e-04 - acc: 4.3365e-04
Epoch 719/1000
 - 0s - loss: 4.3477e-04 - acc: 4.3365e-04
Epoch 720/1000
 - 0s - loss: 4.3490e-04 - acc: 4.3365e-04
Epoch 721/1000
 - 0s - loss: 4.3231e-04 - acc: 4.3365e-04
Epoch 722/1000
 - 0s - loss: 4.2937e-04 - acc: 4.3365e-04
Epoch 723/1000
 - 0s - loss: 4.3522e-04 - acc: 4.3365e-04
Epoch 724/1000
 - 0s - loss: 4.3019e-04 - acc: 4.3365e-04
Epoch 725/1000
 - 0s - loss: 4.3365e-04 - acc: 4.3365e-04
Epoch 726/1000
 - 0s - loss: 4.3142e-04 - acc: 4.3365e-04
Epoch 727/1000
 - 0s - loss: 4.3913e-04 - acc: 4.3365e-04
Epoch 728/1000
 - 0s - loss: 

Epoch 853/1000
 - 0s - loss: 4.3170e-04 - acc: 4.3365e-04
Epoch 854/1000
 - 0s - loss: 4.3393e-04 - acc: 4.3365e-04
Epoch 855/1000
 - 0s - loss: 4.3629e-04 - acc: 4.3365e-04
Epoch 856/1000
 - 0s - loss: 4.3190e-04 - acc: 4.3365e-04
Epoch 857/1000
 - 0s - loss: 4.3426e-04 - acc: 4.3365e-04
Epoch 858/1000
 - 0s - loss: 4.3049e-04 - acc: 4.3365e-04
Epoch 859/1000
 - 0s - loss: 4.4610e-04 - acc: 4.3365e-04
Epoch 860/1000
 - 0s - loss: 4.3727e-04 - acc: 4.3365e-04
Epoch 861/1000
 - 0s - loss: 4.3169e-04 - acc: 4.3365e-04
Epoch 862/1000
 - 0s - loss: 4.3430e-04 - acc: 4.3365e-04
Epoch 863/1000
 - 0s - loss: 4.3515e-04 - acc: 4.3365e-04
Epoch 864/1000
 - 0s - loss: 4.4210e-04 - acc: 4.3365e-04
Epoch 865/1000
 - 0s - loss: 4.3606e-04 - acc: 4.3365e-04
Epoch 866/1000
 - 0s - loss: 4.3430e-04 - acc: 4.3365e-04
Epoch 867/1000
 - 0s - loss: 4.3441e-04 - acc: 4.3365e-04
Epoch 868/1000
 - 0s - loss: 4.3557e-04 - acc: 4.3365e-04
Epoch 869/1000
 - 0s - loss: 4.3040e-04 - acc: 4.3365e-04
Epoch 870/1000

 - 0s - loss: 4.3560e-04 - acc: 4.3365e-04
Epoch 995/1000
 - 0s - loss: 4.3016e-04 - acc: 4.3365e-04
Epoch 996/1000
 - 0s - loss: 4.3094e-04 - acc: 4.3365e-04
Epoch 997/1000
 - 0s - loss: 4.3227e-04 - acc: 4.3365e-04
Epoch 998/1000
 - 0s - loss: 4.3011e-04 - acc: 4.3365e-04
Epoch 999/1000
 - 0s - loss: 4.3114e-04 - acc: 4.3365e-04
Epoch 1000/1000
 - 0s - loss: 4.3186e-04 - acc: 4.3365e-04
Train Score: 543.83 RMSE
Predict the Close value of final day: 17196
